In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index

In [2]:
import xarray
import numpy


In [10]:
from eofs.xarray import Eof

In [5]:
def sstAnoms(iModel):
    #sst
    controlXr = fh.loadModelData(iModel[1], 'ts_Amon', 'piControl', iModel[2])
    tsXr = fh.loadModelData(iModel[1], 'ts_Amon', 'historical', iModel[3])
    tradIndecesXr = xarray.open_dataset('results/cmipMonthlyIndeces/'+iModel[1]+'_historical.nc')

    
    sstMeanXr=controlXr.where(
        (controlXr.lat>-15) & (controlXr.lat<15) &
        (controlXr.lon>140) & (controlXr.lon<280),
        drop=True
        ).ts.groupby('time.month').mean(dim='time')

    sstAnomXr=tsXr.where(
        (tsXr.lat>-15) & (tsXr.lat<15) &
        (tsXr.lon>140) & (tsXr.lon<280),
        drop=True
        ).ts.groupby('time.month'
                )-sstMeanXr-tradIndecesXr['backgroundSstNoDetrend']
    
    return sstAnomXr
    


In [ ]:
def solverDCF(iModel): 
    
    sstAnomXr = sstAnoms(iModel)
    
    sstAnomXr=sstAnomXr.where(
                    (sstAnomXr.month==1) + (sstAnomXr.month==2) + (sstAnomXr.month==12),
                drop=True)
    
    sstAnomXr.load()
    
    weights = numpy.cos(numpy.deg2rad(sstAnomXr.lat)
                ).values[..., numpy.newaxis]
    
    return Eof(sstAnomXr, weights=weights)

In [ ]:
def solver(iModel): 
    
    sstAnomXr = sstAnoms(iModel)
    
    sstAnomXr.load()
    
    weights = numpy.cos(numpy.deg2rad(sstAnomXr.lat)
                ).values[..., numpy.newaxis]
    
    return Eof(sstAnomXr, weights=weights)

In [3]:
def ecIndex(solver):
    import numpy.polynomial as poly
    pcTimeXr=solver.pcs(pcscaling=1, npcs=2)
    pc1 = pcTimeXr.sel(mode=0)
    pc2 = pcTimeXr.sel(mode=1)
    pFit = poly.Polynomial.fit(pc1, pc2, 2)
    alpha = pFit.convert(domain=(-1, 1)).coef[2]
    cXr=(pc1+pc2)/numpy.sqrt(2)
    eXr=(pc1-pc2)/numpy.sqrt(2)
    
    indeces = xarray.merge([eXr.rename('eIndex'),cXr.rename('cIndex')])
    indeces['alpha']=alpha
    
    
    return indeces